In [8]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import math


# 最近は、等分散の検定を行わず、Welchのt検定を行うのが一般的!?
https://qiita.com/suaaa7/items/745ac1ca0a8d6753cf60

In [18]:
A = np.array([13.8, 10.2, 4.6, 10.0, 4.2, 16.1, 14.4, 4.9, 7.7, 11.4])
B = np.array([3.3, 2.6, 4.0, 4.7, 1.9, 2.9, 4.7, 5.3, 4.3, 3.0, 2.0])

In [26]:
# 平均と平均の差を出力する
def hekin(A, B): 
    a_mean = A.mean()
    b_mean = B.mean()
    sa = abs(a_mean - b_mean)
    
    print('Aの平均は{}, Bの平均は{}, \n平均の差は{}'.format(a_mean, b_mean, sa))
hekin(A, B)

Aの平均は9.73, Bの平均は3.5181818181818176, 
平均の差は6.211818181818183


In [19]:
stats.ttest_ind(A, B, equal_var=False)

Ttest_indResult(statistic=4.426442804187721, pvalue=0.0012285738375064346)

p < 0.05なので、帰無仮説は棄却され、2群間には差がある

# このサイトを参考
# https://qiita.com/code0327/items/a96dd2fbd8a491d2eeaa

A組とB組の学生に対して、異なる学習指導法を実施したら、試験成績に影響が出るのか

帰無仮説: 
対立仮説: 2つの学習指導法には、

In [14]:
xa = pd.Series( [75, 87, 89, 80, 84, 81, 88, 83, 88, 88, 82, 72, 74, 93, 77, 67, 88, 84, 68, 84, 80, 78, 75, 71, 82, 74, 84, 77, 79, 76, 83, 75, 86, 76, 80, 76, 68, 72, 75, 85])
xb = pd.Series( [64, 77, 79, 73, 89, 82, 59, 85, 80, 75, 65, 79, 65, 74, 73, 72, 69, 83, 90, 73, 88, 59, 62, 80, 64, 74, 81, 70, 69, 67, 81, 67, 72, 71, 72, 78, 78, 82, 72, 71])

print('A組の平均点 = {}'.format(xa.mean()))
print('B組の平均点 = {}'.format(xb.mean()))
print('2群の平均の差 = {}'.format(abs(xa.mean() - xb.mean())))
print('標本Aの不偏分散 = {}'.format(xa.std(ddof=1)))
print('標本Bの不偏分散 = {}'.format(xb.std(ddof=1)))

A組の平均点 = 79.6
B組の平均点 = 74.1
2群の平均の差 = 5.5
標本Aの不偏分散 = 6.42431279373871
標本Bの不偏分散 = 7.886372537160021


In [13]:
print('標本Aの不偏分散 = {}'.format(xa.std(ddof=1)))
print('標本Bの不偏分散 = {}'.format(xb.std(ddof=1)))

標本Aの不偏分散 = 6.42431279373871
標本Bの不偏分散 = 7.886372537160021


# シャピロ・ウィルク検定による正規性のチェック
「標本は、正規分布に従う母集団から無作為抽出されたものである」を帰無仮説とする

In [16]:
_, pa = stats.shapiro(xa)
_, pb = stats.shapiro(xb)
print('シャピロ・ウィルク検定')
print(f'標本A p値 = {pa:.3f}')
print(f'標本B p値 = {pb:.3f}')

シャピロ・ウィルク検定
標本A p値 = 0.558
標本B p値 = 0.747


-> p > 0.05のため、帰無仮説を採択し、各標本の母集団には正規性がある

# ルビーン検定による等分散性のチェック
「2つの母分散が等しいことを帰無仮説とする」

In [17]:
_, p = stats.levene(xa, xb, center='mean')
print('ルビーン検定')
print(f'p値 = {p:.3f}')

ルビーン検定
p値 = 0.272


-> p>0.05のため、帰無仮説を採択し、2つの標本には等分散性があると判断する

# 母平均の95%信頼区間を求める
# (正規分布に従う場合)

In [15]:
# 母集団の分散推定値
u2 = xa.var(ddof=1)

# 標本平均
m = xa.mean()

# 自由度
DF = len(xa) - 1

# 標準誤差
SE = math.sqrt(u2 / len(xa))

CI1,CI2 = stats.t.interval( alpha=0.95, loc=m, scale=SE, df=DF )
print(f'標本Aの母平均の95%信頼区間CI = [{CI1:.2f} , {CI2:.2f}]')

標本Aの母平均の95%信頼区間CI = [77.55 , 81.65]


# このサイトを参考
# https://logics-of-blue.com/t-test/

In [2]:
data = pd.DataFrame({
    'X': [2, 0, 3, -3, 4, 1, -1, 4], 
    'Y': [5, -1, 2, -1, 7, 3, 4, 5]
})
data

,X,Y
0,2,5
1,0,-1
2,3,2
3,-3,-1
4,4,7
5,1,3
6,-1,4
7,4,5


# 1群のt検定

In [3]:
stats.ttest_1samp(data['X'], 0)

Ttest_1sampResult(statistic=1.4182715723279384, pvalue=0.19905953953787844)

-> p > 0.05のため、平均値は0と異なるとは言えない

# 対応のあるt検定

In [4]:
stats.ttest_rel(data['X'], data['Y'])

Ttest_relResult(statistic=-2.4111323733951022, pvalue=0.04669626885950978)

-> p値 < 0.05のため、XとYで、平均値は異なる

# 対応のない検定（分散が等しい）

In [5]:
stats.ttest_ind(data['X'], data['Y'], equal_var = True)

Ttest_indResult(statistic=-1.299867367239363, pvalue=0.21464181738984914)

-> p値 > 0.05のため、XとYで平均値は異ならない

# 対応のない検定（分散が異なる）

In [6]:
stats.ttest_ind(data['X'], data['Y'], equal_var = False)

Ttest_indResult(statistic=-1.299867367239363, pvalue=0.21506175265308078)

-> p値 > 0.05のため、XとYで平均値は異ならない